In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-Instruct-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, torch_dtype=torch.float16, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "textminr/mistral-7b-4bit-ner")

In [3]:
eval_prompt= """### Question: Der süße Brei
Ein Märchen der Gebrüder Grimm gestaltet von Heike Weinbrecht und erschienen am 27. Jänner 1830: Es war einmal ein armes, frommes Mädchen, das lebte mit seiner Mutter allein, und sie hatten nichts mehr zu essen.\n### Answer:"""

In [15]:
eval_prompt2= """### Question: A recent editorial discusses the ongoing challenges in global economic policies. The article delves into various aspects of international trade agreements, monetary policies, and the impacts of globalization. It draws comparisons between current economic trends and those of the last decade, offering insights into potential future developments.\n### Answer:"""

In [18]:
eval_prompt3 = """### Question: An insightful analysis of Shakespeare's plays, focusing on the thematic use of ambition in 'Macbeth'. The essay, published in 2005, examines the play's characters and their motivations, drawing parallels to modern societal structures. The intricate dissection of dialogue and stage direction offers a fresh perspective on the Bard's timeless work.\n### Answer:"""

In [20]:
eval_prompt4 = """### Question: In his book 'The Fabric of the Cosmos', Brian Greene explores the fundamental elements of the universe, from quantum mechanics to the nature of space and time. Throughout the book, Greene elucidates complex theories with accessible language, making sophisticated concepts in physics comprehensible to the general reader.\n### Answer:"""

In [21]:
# eval_prompt = "### Question: In their groundbreaking 2020 study on climate change impacts, Smith and Jones (2020) highlight the urgent need for sustainable practices.\n### Answer:"
model_input = tokenizer(eval_prompt4, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=100, repetition_penalty=1.15)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### Question: In his book 'The Fabric of the Cosmos', Brian Greene explores the fundamental elements of the universe, from quantum mechanics to the nature of space and time. Throughout the book, Greene elucidates complex theories with accessible language, making sophisticated concepts in physics comprehensible to the general reader.
### Answer: {"author": "Brian Greene", "date": "N/A"}
